<a href="https://colab.research.google.com/github/NNL-Keerthana/DeepLearning_EEG/blob/main/EEGnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
dataset_path = '/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0'
os.chdir(dataset_path)
#/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv

In [ ]:
import mne
import numpy as np
import pandas as pd

# Load subject info from subject-info.csv
subject_info = pd.read_csv('/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv')

# Function to calculate band-wise PSD
def calculate_band_psd(raw, sfreq, freq_bands):
    psds, freqs = mne.time_frequency.psd_array_multitaper(raw.get_data(), sfreq=sfreq, fmin=0.5, fmax=40, adaptive=True, normalization='full', verbose=0)
    band_psd = dict()
    for band, (fmin, fmax) in freq_bands.items():
        band_psd[band] = np.mean(psds[:, (freqs >= fmin) & (freqs < fmax)], axis=1)
    return band_psd

# Define frequency bands of interest
freq_bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 30),
    'Gamma': (30, 40)
}

# Function to load and preprocess data
def load_and_save_eeg_data(subject_info, freq_bands):
    X_list = []
    y_list = []
    for idx, row in subject_info.iterrows():
        subject_id = row['Subject']  # Directly use 'Subject' column from your CSV
        # File paths
        file_rest = f'/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/{subject_id}_1.edf'
        file_task = f'/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/{subject_id}_2.edf'

        # Load EDF files
        raw_rest = mne.io.read_raw_edf(file_rest, preload=True)
        raw_task = mne.io.read_raw_edf(file_task, preload=True)

        # Filter raw data
        raw_rest.filter(1, 40, fir_design='firwin')
        raw_task.filter(1, 40, fir_design='firwin')

        # Calculate PSD
        psd_rest = calculate_band_psd(raw_rest, raw_rest.info['sfreq'], freq_bands)
        psd_task = calculate_band_psd(raw_task, raw_task.info['sfreq'], freq_bands)

        # Flatten PSD values for machine learning input
        X_rest = np.concatenate([psd_rest[band] for band in freq_bands.keys()])
        X_task = np.concatenate([psd_task[band] for band in freq_bands.keys()])

        # Labels: 0 for rest, 1 for task
        y_rest = np.zeros(X_rest.shape[0])
        y_task = np.ones(X_task.shape[0])

        # Concatenate features and labels
        X_list.append(X_rest)
        X_list.append(X_task)
        y_list.append(y_rest)
        y_list.append(y_task)

    X = np.array(X_list)
    y = np.array(y_list)
    np.save('X_psd.npy', X)
    np.save('y_psd.npy', y)
    print(f"Saved PSD features to 'X_psd.npy' and labels to 'y_psd.npy'")

# Run the function to calculate and save PSD features
load_and_save_eeg_data(subject_info, freq_bands)


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

...

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Saved PSD features to 'X_psd.npy' and labels to 'y_psd.npy'


# EEGNET

Preprocessing

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, DepthwiseConv2D, AveragePooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.constraints import max_norm

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# Ensure the data is reshaped appropriately for EEGNet
print(f"Original X shape: {X.shape}")
print(f"Original y shape: {y.shape}")

# EEGNet expects 4D input: (samples, 1, channels, 1)
X = X.reshape(X.shape[0], 1, X.shape[1], 1)
print(f"Reshaped X shape: {X.shape}")

# Ensure y is a 1D array before one-hot encoding
if y.ndim > 1:
    y = y[:, 0]
print(f"Corrected y shape: {y.shape}")

# One-hot encode labels (this part remains the same)
y = to_categorical(y)
print(f"One-hot encoded y shape: {y.shape}")

# Flatten X for the train_test_split
X_flat = X.reshape(X.shape[0], -1)
print(f"Flattened X shape: {X_flat.shape}")

# Use argmax to get 1D array of labels for stratification
y_for_stratify = np.argmax(y, axis=1)  # Get the index of the class with highest probability

# Split into train and test sets (use y_for_stratify for stratification)
X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.2, random_state=42, stratify=y_for_stratify
)

# Reshape back to original dimensions after the split
X_train = X_train_flat.reshape(X_train_flat.shape[0], 1, -1, 1)
X_test = X_test_flat.reshape(X_test_flat.shape[0], 1, -1, 1)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Normalize the data
scaler = StandardScaler()

# Flatten the training data, fit the scaler, and reshape back to the original shape
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_train_scaled = scaler.fit_transform(X_train_flat)
X_train = X_train_scaled.reshape(X_train.shape)

# Flatten the testing data, apply the scaler, and reshape back to the original shape
X_test_flat = X_test.reshape(X_test.shape[0], -1)
X_test_scaled = scaler.transform(X_test_flat)
X_test = X_test_scaled.reshape(X_test.shape)

print(f"Normalized X_train shape: {X_train.shape}")
print(f"Normalized X_test shape: {X_test.shape}")


Original X shape: (72, 105)
Original y shape: (72, 105)
Reshaped X shape: (72, 1, 105, 1)
Corrected y shape: (72,)
One-hot encoded y shape: (72, 2)
Flattened X shape: (72, 105)
X_train shape: (57, 1, 105, 1)
X_test shape: (15, 1, 105, 1)
Normalized X_train shape: (57, 1, 105, 1)
Normalized X_test shape: (15, 1, 105, 1)


## Initial Model

In [ ]:
# EEGNet model definition
def EEGNet(nb_classes, Chans=64, Samples=128, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25):
    model = Sequential()
    model.add(Conv2D(F1, (1, kernLength), padding='same', input_shape=(1, Samples, 1), use_bias=False))
    model.add(BatchNormalization())
    model.add(DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D, depthwise_constraint=max_norm(1.)))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((1, 4)))
    model.add(Dropout(dropoutRate))

    model.add(Flatten())
    model.add(Dense(nb_classes, kernel_constraint=max_norm(norm_rate)))
    model.add(Activation('softmax'))

    return model

In [ ]:
# Define model with adjusted Chans parameter to match your input shape
model = EEGNet(nb_classes=2, Chans=1, Samples=X.shape[2], dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, batch_size=16, epochs=70, verbose=2, validation_split=0.2)

# Evaluate model
score = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {score[1] * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
3/3 - 9s - 3s/step - accuracy: 0.4444 - loss: 0.9024 - val_accuracy: 0.7500 - val_loss: 0.6828
Epoch 2/70
3/3 - 0s - 16ms/step - accuracy: 0.6222 - loss: 0.6866 - val_accuracy: 0.7500 - val_loss: 0.6720
Epoch 3/70
3/3 - 0s - 15ms/step - accuracy: 0.6444 - loss: 0.6451 - val_accuracy: 0.7500 - val_loss: 0.6651
Epoch 4/70
3/3 - 0s - 15ms/step - accuracy: 0.7111 - loss: 0.6117 - val_accuracy: 0.8333 - val_loss: 0.6644
Epoch 5/70
3/3 - 0s - 18ms/step - accuracy: 0.7111 - loss: 0.5847 - val_accuracy: 0.9167 - val_loss: 0.6620
Epoch 6/70
3/3 - 0s - 14ms/step - accuracy: 0.7778 - loss: 0.5707 - val_accuracy: 0.8333 - val_loss: 0.6596
Epoch 7/70
3/3 - 0s - 15ms/step - accuracy: 0.7778 - loss: 0.5341 - val_accuracy: 0.8333 - val_loss: 0.6573
Epoch 8/70
3/3 - 0s - 15ms/step - accuracy: 0.8444 - loss: 0.5151 - val_accuracy: 0.8333 - val_loss: 0.6526
Epoch 9/70
3/3 - 0s - 15ms/step - accuracy: 0.8222 - loss: 0.5071 - val_accuracy: 0.9167 - val_loss: 0.6478
Epoch 10/70
3/3 - 0s - 21ms/st

In [ ]:
# Get predicted probabilities from the model
y_pred_proba = model.predict(X_test)

# Convert predicted probabilities to class labels (0 or 1)
y_pred = np.argmax(y_pred_proba, axis=1)

# Convert y_test to 1D array of class labels
y_test_labels = np.argmax(y_test, axis=1)

# Now you can calculate the metrics
accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')
f1 = f1_score(y_test_labels, y_pred, average='weighted')

# Print evaluation metrics
print(f"Test accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
Test accuracy: 66.67%
Precision: 0.67
Recall: 0.67
F1-score: 0.67




---



## 2) Improved Model with changes in Hyperparameters.

Accuracy 66.67% to 73.33%

Explored with dropoutRate, normRate and Depth

In [ ]:
# EEGNet model definition
def EEGNet(nb_classes, Chans=64, Samples=128, dropoutRate=0.3, kernLength=64, F1=8, D=8, F2=16, norm_rate=0.5):
    model = Sequential()
    model.add(Conv2D(F1, (1, kernLength), padding='same', input_shape=(1, Samples, 1), use_bias=False))
    model.add(BatchNormalization())
    model.add(DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D, depthwise_constraint=max_norm(1.)))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((1, 4)))
    model.add(Dropout(dropoutRate))

    model.add(Flatten())
    model.add(Dense(nb_classes, kernel_constraint=max_norm(norm_rate)))
    model.add(Activation('softmax'))

    return model

In [ ]:
from tensorflow.keras.optimizers import Adam

# Define model with adjusted Chans parameter to match your input shape
model = EEGNet(nb_classes=2, Chans=1, Samples=X.shape[2], dropoutRate=0.3, kernLength=64, F1=8, D=8, F2=16, norm_rate=0.5)

optimizer = Adam(learning_rate=0.0001)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, batch_size=16, epochs=50, verbose=2, validation_split=0.2)

# Evaluate model
score = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {score[1] * 100:.2f}%")

Epoch 1/50
3/3 - 4s - loss: 0.8939 - accuracy: 0.3333 - val_loss: 0.6797 - val_accuracy: 0.7500 - 4s/epoch - 1s/step
Epoch 2/50
3/3 - 0s - loss: 0.6737 - accuracy: 0.6222 - val_loss: 0.6610 - val_accuracy: 0.8333 - 78ms/epoch - 26ms/step
Epoch 3/50
3/3 - 0s - loss: 0.5468 - accuracy: 0.7333 - val_loss: 0.6545 - val_accuracy: 0.9167 - 83ms/epoch - 28ms/step
Epoch 4/50
3/3 - 0s - loss: 0.5865 - accuracy: 0.7778 - val_loss: 0.6264 - val_accuracy: 0.9167 - 60ms/epoch - 20ms/step
Epoch 5/50
3/3 - 0s - loss: 0.4780 - accuracy: 0.7778 - val_loss: 0.6160 - val_accuracy: 0.9167 - 56ms/epoch - 19ms/step
Epoch 6/50
3/3 - 0s - loss: 0.4593 - accuracy: 0.7778 - val_loss: 0.6151 - val_accuracy: 0.7500 - 69ms/epoch - 23ms/step
Epoch 7/50
3/3 - 0s - loss: 0.3945 - accuracy: 0.8222 - val_loss: 0.6239 - val_accuracy: 0.8333 - 58ms/epoch - 19ms/step
Epoch 8/50
3/3 - 0s - loss: 0.3946 - accuracy: 0.7778 - val_loss: 0.6300 - val_accuracy: 0.7500 - 79ms/epoch - 26ms/step
Epoch 9/50
3/3 - 0s - loss: 0.3468 -

Referred EEGNET Docs: https://github.com/vlawhern/arl-eegmodels/tree/master


Model Performance tips: https://journals.sagepub.com/doi/full/10.26599/BSA.2022.9050007


---


Best Models:

## 3) 100% Precision (Accuracy 73.33%)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, DepthwiseConv2D, Activation, AveragePooling2D, Dropout, SeparableConv2D, Flatten, Dense
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def EEGNet(nb_classes, Chans=64, Samples=128, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout'):
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout, passed as a string.')

    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same', input_shape=(Chans, Samples, 1), use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D, depthwise_constraint=max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = dropoutType(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = dropoutType(dropoutRate)(block2)

    flatten = Flatten(name='flatten')(block2)

    dense = Dense(nb_classes, name='dense', kernel_constraint=max_norm(norm_rate))(flatten)
    softmax = Activation('softmax', name='softmax')(dense)

    return Model(inputs=input1, outputs=softmax)


In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# Ensure the data is reshaped appropriately for EEGNet
print(f"Original X shape: {X.shape}")
print(f"Original y shape: {y.shape}")

# EEGNet expects 4D input: (samples, channels, time points, 1)
X = X.reshape(X.shape[0], 1, X.shape[1], 1)
print(f"Reshaped X shape: {X.shape}")

# Ensure y is a 1D array before one-hot encoding
if y.ndim > 1:
    y = y[:, 0]
print(f"Corrected y shape: {y.shape}")

# One-hot encode labels
y = to_categorical(y)
print(f"One-hot encoded y shape: {y.shape}")

# Flatten X for the train_test_split
X_flat = X.reshape(X.shape[0], -1)
print(f"Flattened X shape: {X_flat.shape}")

# Use argmax to get 1D array of labels for stratification
y_for_stratify = np.argmax(y, axis=1)

# Split into train and test sets (use y_for_stratify for stratification)
X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.2, random_state=42, stratify=y_for_stratify
)

# Reshape back to original dimensions after the split
X_train = X_train_flat.reshape(X_train_flat.shape[0], 1, -1, 1)
X_test = X_test_flat.reshape(X_test_flat.shape[0], 1, -1, 1)

# Normalize the data
scaler = StandardScaler()

# Flatten the training data, fit the scaler, and reshape back to the original shape
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_train_scaled = scaler.fit_transform(X_train_flat)
X_train = X_train_scaled.reshape(X_train.shape)

# Flatten the testing data, apply the scaler, and reshape back to the original shape
X_test_flat = X_test.reshape(X_test.shape[0], -1)
X_test_scaled = scaler.transform(X_test_flat)
X_test = X_test_scaled.reshape(X_test.shape)

# Ensure reshaped data matches the expected input shape for EEGNet
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Define and compile the EEGNet model
model = EEGNet(nb_classes=2, Chans=1, Samples=X.shape[2], dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout')
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=16, epochs=50, verbose=2, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=-1)
y_true = y_test.argmax(axis=-1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes)

print(f"Test accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")


Original X shape: (72, 105)
Original y shape: (72, 105)
Reshaped X shape: (72, 1, 105, 1)
Corrected y shape: (72,)
One-hot encoded y shape: (72, 2)
Flattened X shape: (72, 105)
X_train shape: (57, 1, 105, 1)
X_test shape: (15, 1, 105, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
3/3 - 13s - 4s/step - accuracy: 0.4889 - loss: 0.8226 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 2/50
3/3 - 0s - 16ms/step - accuracy: 0.5556 - loss: 0.6687 - val_accuracy: 0.5833 - val_loss: 0.6929
Epoch 3/50
3/3 - 0s - 14ms/step - accuracy: 0.6000 - loss: 0.6750 - val_accuracy: 0.6667 - val_loss: 0.6928
Epoch 4/50
3/3 - 0s - 15ms/step - accuracy: 0.5778 - loss: 0.6858 - val_accuracy: 0.7500 - val_loss: 0.6928
Epoch 5/50
3/3 - 0s - 16ms/step - accuracy: 0.5333 - loss: 0.6799 - val_accuracy: 0.8333 - val_loss: 0.6927
Epoch 6/50
3/3 - 0s - 16ms/step - accuracy: 0.5333 - loss: 0.6871 - val_accuracy: 0.8333 - val_loss: 0.6926
Epoch 7/50
3/3 - 0s - 20ms/step - accuracy: 0.7111 - loss: 0.6648 - val_accuracy: 0.8333 - val_loss: 0.6925
Epoch 8/50
3/3 - 0s - 21ms/step - accuracy: 0.5556 - loss: 0.6962 - val_accuracy: 0.7500 - val_loss: 0.6924
Epoch 9/50
3/3 - 0s - 44ms/step - accuracy: 0.5111 - loss: 0.6969 - val_accuracy: 0.7500 - val_loss: 0.6923
Epoch 10/50
3/3 - 0s - 44ms/s

## 4) Best Model for EEG medical purposes

100% Recall. 80% Accuracy

Explored with different epochs and tweaks in EEGNet Architecture

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, DepthwiseConv2D, BatchNormalization, Activation, Dropout, Flatten, Dense

# Assuming EEGNet architecture is defined as follows (you may need to adjust this based on your specific EEGNet implementation)
def EEGNet(nb_classes, Chans=64, Samples=128, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout'):
    input1 = Input(shape=(1, Samples, Chans))

    block1 = Conv2D(F1, (1, kernLength), padding='same', input_shape=(1, Samples, Chans), use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D, depthwise_initializer='glorot_uniform')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    flatten = Flatten(name='flatten')(block2)

    dense = Dense(nb_classes, name='out', kernel_constraint='max_norm')(flatten)
    softmax = Activation('softmax', name='softmax')(dense)

    return Model(inputs=input1, outputs=softmax)

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# Ensure the data is reshaped appropriately for EEGNet
X = X.reshape(X.shape[0], 1, X.shape[1], 1)

# Ensure y is a 1D array before one-hot encoding
if y.ndim > 1:
    y = y[:, 0]

# One-hot encode labels
y = to_categorical(y)

# Flatten X for the train_test_split
X_flat = X.reshape(X.shape[0], -1)

# Use argmax to get 1D array of labels for stratification
y_for_stratify = np.argmax(y, axis=1)

# Split into train and test sets (use y_for_stratify for stratification)
X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.2, random_state=42, stratify=y_for_stratify
)

# Reshape back to original dimensions after the split
X_train = X_train_flat.reshape(X_train_flat.shape[0], 1, -1, 1)
X_test = X_test_flat.reshape(X_test_flat.shape[0], 1, -1, 1)

# Normalize the data
scaler = StandardScaler()

# Flatten the training data, fit the scaler, and reshape back to the original shape
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_train_scaled = scaler.fit_transform(X_train_flat)
X_train = X_train_scaled.reshape(X_train.shape)

# Flatten the testing data, apply the scaler, and reshape back to the original shape
X_test_flat = X_test.reshape(X_test.shape[0], -1)
X_test_scaled = scaler.transform(X_test_flat)
X_test = X_test_scaled.reshape(X_test.shape)

# Ensure reshaped data matches the expected input shape for EEGNet
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")



X_train shape: (57, 1, 105, 1)
X_test shape: (15, 1, 105, 1)


In [ ]:
# Define and compile the EEGNet model
model = EEGNet(nb_classes=2, Chans=1, Samples=X.shape[2], dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout')
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, batch_size=16, epochs=70, verbose=2, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = np.argmax(y_test, axis=-1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes)

print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Epoch 1/70


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 - 15s - 5s/step - accuracy: 0.4889 - loss: 0.7958 - val_accuracy: 0.2500 - val_loss: 0.7047
Epoch 2/70
3/3 - 9s - 3s/step - accuracy: 0.5111 - loss: 0.7704 - val_accuracy: 0.3333 - val_loss: 0.7044
Epoch 3/70
3/3 - 0s - 18ms/step - accuracy: 0.4889 - loss: 0.8771 - val_accuracy: 0.3333 - val_loss: 0.7040
Epoch 4/70
3/3 - 0s - 19ms/step - accuracy: 0.6222 - loss: 0.7153 - val_accuracy: 0.3333 - val_loss: 0.7037
Epoch 5/70
3/3 - 0s - 16ms/step - accuracy: 0.4222 - loss: 0.9141 - val_accuracy: 0.3333 - val_loss: 0.7035
Epoch 6/70
3/3 - 0s - 17ms/step - accuracy: 0.4222 - loss: 0.8628 - val_accuracy: 0.3333 - val_loss: 0.7032
Epoch 7/70
3/3 - 0s - 16ms/step - accuracy: 0.5111 - loss: 0.8265 - val_accuracy: 0.3333 - val_loss: 0.7028
Epoch 8/70
3/3 - 0s - 16ms/step - accuracy: 0.5778 - loss: 0.7265 - val_accuracy: 0.3333 - val_loss: 0.7024
Epoch 9/70
3/3 - 0s - 20ms/step - accuracy: 0.4444 - loss: 0.7718 - val_accuracy: 0.3333 - val_loss: 0.7020
Epoch 10/70
3/3 - 0s - 18ms/step - accurac

## Inference:
In medical diagnostics where missing a positive case is critical higher Recall is preferred. Hence this Model with 100% Recall is suitable.
Moreover, the other metrics are good and decently optimised
accuracy of 80%, precision at 70% and F1 score of 0.8235

